### 1. Import Packages

In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

print('All packages successfully loaded')

All packages successfully loaded


### 2. Load Data & Peak Sheet

In [18]:
df = pd.read_excel("/Users/nova/Desktop/Thesis/Metabolomics ML/data/MTBLS90.xlsx", index_col="Idx")
df.head()

/Users/nova/Programming/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,SampleID,Class,Sex,M1,M2,M3,M4,M5,M6,M7,...,M180,M181,M182,M183,M184,M185,M186,M187,M188,M189
Idx,,,,,,,,,,,,,,,,,,,,,
1,subject_1524,0,Female,14.98474,14.49947,12.097435,12.043818,11.436392,16.07217,17.01090,...,20.70097,20.19814,21.28179,21.09176,16.46404,12.269896,17.29323,19.81546,19.35259,12.71608
2,subject_1525,1,Male,15.17667,14.50172,12.363773,12.240458,11.577436,14.93265,16.90847,...,21.68605,20.51608,21.25777,20.53935,16.33413,12.031267,18.08144,19.96693,19.50694,13.20891
3,subject_1527,1,Male,15.35934,14.41941,12.510494,11.234959,11.199970,15.22600,17.17192,...,20.46320,20.34297,21.34793,20.81883,16.44281,12.679043,17.24765,20.01505,20.34058,12.98210
4,subject_1528,1,Male,15.12638,14.75630,13.001426,12.640496,11.244146,14.78994,16.47827,...,21.36879,20.93957,21.09719,20.82021,16.27483,12.323163,17.72392,19.57522,19.18157,12.41039
5,subject_1529,1,Male,15.20272,14.48083,12.392280,10.779516,10.968423,15.75598,17.20552,...,20.80482,19.82493,20.84250,20.31349,16.33279,12.352881,17.59348,19.88594,18.66284,12.37479


### 3. Extract X & Y

In [19]:
X = df.drop(columns=['SampleID', 'Sex', 'Class'])
y = df.Class
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 3.1 Data Cleaning

In [20]:
# Calcolo il numero di valori mancanti per ogni colonna
missing_values = X_train.isnull().sum()
# Sommo i valori della serie ottenuta
number_missing = missing_values.sum()
# Calcolo il numero totale di celle
total_cells = np.product(X_train.shape)
# Calcolo la percentuale di valori mancanti
percent_missing = (number_missing/total_cells) * 100
print("Percentuale di dati mancanti: {:.1f}%".format(percent_missing))
print("Dimensioni del dataframe originale: " + str(X_train.shape))

# Droppo le righe con valori mancanti
X_train.dropna(axis=0, inplace=True)
y_train.dropna(axis=0, inplace=True)
print("----------------------------------------------")

# Effettuo le stesse operazioni dopo aver droppato le righe con valori mancanti
missing_values = X_train.isnull().sum()
number_missing = missing_values.sum()
total_cells = np.product(X_train.shape)
percent_missing = (number_missing/total_cells) * 100
print("Percentuale di dati mancanti: {:.1f}%".format(percent_missing))
print("Dimensioni del nuovo dataframe: " + str(X_train.shape))

Percentuale di dati mancanti: 0.0%
Dimensioni del dataframe originale: (774, 189)
----------------------------------------------
Percentuale di dati mancanti: 0.0%
Dimensioni del nuovo dataframe: (774, 189)


In sintesi non ci sono dati valori all'interno del dataset, perciò l'ultima cella è pressoché inutile.

### 4. Hyperparameters Optimisation

In [21]:
# Qui di seguito effetuiamo il tuning degli iperparametri del modello

# Creiamo il modelo XGBClassifier
model = xgb.XGBClassifier()

# GRID SEARCH

# Definiamo la griglia con i parametri da testare

"""" IPERPARAMETRI ORIGINALI (il mio computer non è riuscito a reggere la complessità)
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'colsample_bylevel': [0.8, 0.9, 1.0],
    'scale_pos_weight': [1, 2, 3]
}
"""

# IPERPARAMETRI RIDOTTI
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 4],
    'min_child_weight': [1, 2],
    'gamma': [0, 0.1],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9],
    'colsample_bylevel': [0.8, 0.9],
    'scale_pos_weight': [1, 2]
}


# Creiamo il modello GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Eseguiamo la Grid Search sui dati di addestramento
grid_search.fit(X_train, y_train)

# Visualizziamo i migliori iperparametri trovati
best_params = grid_search.best_params_
print("Migliori iperparametri:", best_params)

# Addestriamo il modello con i migliori iperparametri
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Valutiamo il modello sui dati di test
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuratezza del modello: {accuracy * 100:.2f}%')


KeyboardInterrupt: 

Dopo 6 minuti, siamo riusciti a ottenere questi risultati: 
Migliori iperparametri: {'colsample_bylevel': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 200, 'scale_pos_weight': 1, 'subsample': 0.8}
Accuratezza del modello: 78.35%

### 5. Build Model & Evaluate

In [ ]:
# Definiamo il modello XGBoost con gli iperparametri ottimizzati
best_model = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=4,
    min_child_weight=2,
    gamma=0.1,
    subsample=0.8,
    colsample_bytree=0.9,
    colsample_bylevel=0.9,
    scale_pos_weight=1
)

# Addestriamo il modello sul set di addestramento completo
best_model.fit(X_train, y_train)

# Valutiamo il modello su X_test
predictions = best_model.predict(X_test)

# Calcoliamo i punteggi del modello
accuracy = accuracy_score(y_test, predictions)
print(f'Accuratezza del modello: {accuracy * 100:.2f}%')

Accuratezza del modello: 78.87%
